In [ ]:
# instructions to download miniconda3 per their website

# from the terminal run these lines of code:
"""
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh

source ~/miniconda3/bin/activate
conda init --all
source ~/.bashrc
"""

In [ ]:
# set up MetaXcan

# clone the repository
"""
cd ~
git clone https://github.com/hakyimlab/MetaXcan.git
"""

# create a conda environment for MetaXcan
"""
cd MetaXcan/software
conda env create -f conda_env.yaml
conda activate imlabtools
"""

### IGNORE BELOW IF TESTING ON COMMAND LINE ###

# then, open up a jupyter notebook
# click on the 'Kernel' tab at the top
# hover over 'Change kernel'
# click on 'Python [conda env:imlabtools]'

In [ ]:
# get covariance matrices via

"""
mkdir allofus_test
cd allofus_test
wget https://zenodo.org/record/3518299/files/mashr_eqtl.tar?download=1 -O mashr_eqtl.tar
tar -xvpf mashr_eqtl.tar
"""

# transcriptome prediction model database and snp covariance matrices contained therein

In [ ]:
# format data for MetaXcan

import os
os.chdir("/home/jupyter/packages/MetaXcan/software/allofus_test")

os.system("gsutil cp gs://fc-secure-bb61452f-d5e2-4d26-9227-6a9444241af8/data/filtered_261.2.tsv .")

os.system("head -n 50001 filtered_261.2.tsv > first50k.tsv")

infile = "first50k.tsv"

import pandas as pd
df = pd.read_csv(infile, sep="\t")
print(df["alleles"])

ref = []
alt = []
for row in df["alleles"]:
    allele_split = row.split(",")
    ref.append(allele_split[0][2:-1])
    alt.append(allele_split[1][1:-2])

df["ref"] = ref
df["alt"] = alt
df.drop("alleles", axis=1, inplace=True)

print(df.head())
print(df.tail())

df.to_csv("allofus_test.tsv", sep="\t")

os.mkdir("data")
os.system("mv allofus_test.tsv data")

In [ ]:
# run S-PrediXcan with test data

import os

### RUN BELOW FROM THE COMMAND LINE ###

os.chdir("/home/jupyter/packages/MetaXcan/software")

os.system("conda activate imlabtools")

os.system("./SPrediXcan.py \
--model_db_path allofus_test/eqtl/mashr/mashr_Stomach.db \
--covariance allofus/test/eqtl/mashr/mashr_Stomach.txt.gz \
--gwas_folder allofus_test/data \
--gwas_file_pattern '*.tsv' \
--snp_column locus \
--effect_allele_column ref \
--non_effect_allele_column alt \
--beta_column BETA \
--pvalue_column Pvalue \
--se_column SE \
--output_file results/test.csv")